#Clustering Task for the Dataset

#### CSV files are obtained from the directory, necessary imports are done.

---





In [ ]:
import pandas as pd
import os
from google.colab import drive
import pandas as pd
import numpy as np
import random

import copy
import matplotlib.pyplot as plt
import time
drive.mount('/content/drive')
MY_DRIVE_PATH = "/content/drive/MyDrive/MLProject_2"
DATA_FOLDER = os.path.join(MY_DRIVE_PATH, 'Data google sheet')
PROCESSED_CSV_FILE = os.path.join(DATA_FOLDER, 'Processed_Fruits_Data.csv')
ONEHOT_CSV_FILE = os.path.join(DATA_FOLDER, 'One_Hot_Processed_Fruits_Data.csv')
PCA_CSV_PATH = os.path.join(DATA_FOLDER, 'PCA_Processed_Fruits.csv')


Mounted at /content/drive


#### Prepare dataset for clustering



In [ ]:
df = pd.read_csv(ONEHOT_CSV_FILE, sep = ";")
np.random.seed(42)
# Clean up columns (Remove non-numeric metadata)
df.drop(columns=["Image_path", "Text", "Label"], inplace=True)

# Identify columns for normalization
numerical_cols = ["Weight", "Price"]
image_cols = [col for col in df.columns if "img" in col]
text_cols = [col for col in df.columns if "text" in col]

# Normalizing every image histogram
for idx in range(len(df)):
    hist_values = df.loc[idx, image_cols].values.astype(np.float64)
    total = hist_values.sum()
    if total > 0:
        df.loc[idx, image_cols] = hist_values / total
    else:
        df.loc[idx, image_cols] = 0

# Combined list of all feature columns (excluding the 'Fruit' target)
columns_to_normalize = numerical_cols + image_cols + text_cols

# Apply Normalization to the whole DataFrame
epsilon = 1e-8
for column in columns_to_normalize:
    mean = df[column].mean()
    std = df[column].std()
    df[column] = (df[column] - mean) / (std + epsilon)

# Prepare data for Clustering
# We drop 'Fruit' because K-means shouldn't see the answer
X_clustering_raw = df.drop(columns=['Fruit']).to_numpy()

# We keep the labels separately for the External Metric (Rand Index) later
y_true_labels = df['Fruit'].values

print(f"Data ready for clustering without PCA. Shape: {X_clustering_raw.shape}")

Data ready for clustering without PCA. Shape: (3182, 492)


#### Clustering Algorithm
We used the K-means++ algorithm for clustering. First, we tried generic k-means, but we experienced the effect of the initial centroids problem, so we decided to add a probabilistic selection of initial centroids part before the generic k-means algorithm part.

In [ ]:
def kmeans_scratch_plusplus(X, k, max_iters=100, tol=1e-4):
    np.random.seed(42)
    n_samples, n_features = X.shape


    #first centroid
    centroids = [X[np.random.randint(n_samples)]]

    for _ in range(1, k):
        # distances from all points to the nearest existing centroid
        dist_sq = np.array([min([np.linalg.norm(x - c)**2 for c in centroids]) for x in X])

        #probabilities
        probs = dist_sq / dist_sq.sum()
        cumulative_probs = np.cumsum(probs)
        #next centroid
        r = np.random.rand()
        for j, p in enumerate(cumulative_probs):
            if r < p:
                centroids.append(X[j])
                break
    centroids = np.array(centroids)
    #K-Means Loop
    for i in range(max_iters):
        # assignment
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        #update
        new_centroids = np.array([X[labels == j].mean(axis=0) if len(X[labels == j]) > 0
                                  else centroids[j] for j in range(k)])

        if np.all(np.abs(new_centroids - centroids) < tol):
            break
        centroids = new_centroids

    return centroids, labels

Since we know that there should be 5 clusters since we have 5 class, we directly give 5 to k.

In [ ]:
X_clustering = df.drop(columns=['Fruit']).to_numpy()

k = 5
centroids, labels = kmeans_scratch_plusplus(X_clustering, k)
print("Clustering is ended. Assigned", len(labels), "points to",k ,"clusters.")

Clustering is ended. Assigned 3182 points to 5 clusters.


#### Internal Metric: ***Silhouette Score***
Reference: ***https://www.geeksforgeeks.org/machine-learning/what-is-silhouette-score/***

We searched for internal metrics, and the first one we saw was the ***silhouette score***, which is intuitive and easy to understand.
With this score, we mainly try to minimize:

***Intra-cluster distance***: The average distance between point $i$ and all other points in the same cluster.

and try to maximize:

***Nearest-cluster distance***: The average distance between point $i$ and all points in the nearest neighboring cluster.

The metric gives the difference of these two; having bigger number as a metric is a good indication for the dataset.

In [ ]:
# For intrinsic metric, we will measure how similar an object is to its own cluste compared to other clusters
# This is called as Silhouette score in the literature.
def silhouette(X, labels):
    n_samples = X.shape[0]
    unique_labels = np.unique(labels)

    #pairwise  Euclidean distance matrix
    dist_matrix = np.sqrt(((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2).sum(axis=2))

    silhouettes = []
    for i in range(n_samples):
        curr_label = labels[i]

        # Average distance to other points in the same cluster
        same_cluster_idx = np.where(labels == curr_label)[0]
        avg_dist_same_cluster_i = np.mean(dist_matrix[i, same_cluster_idx[same_cluster_idx != i]])

        #Average distance to points in the closest other cluster
        avg_dist_other_clusters_i = np.inf
        for other_label in unique_labels:
            if other_label == curr_label:
                continue
            other_cluster_idx = np.where(labels == other_label)[0]
            avg_dist_other = np.mean(dist_matrix[i, other_cluster_idx])
            avg_dist_other_clusters_i = min(avg_dist_other_clusters_i, avg_dist_other)

        # Silhouette for this specific point
        s_i = (avg_dist_other_clusters_i - avg_dist_same_cluster_i) / max(avg_dist_same_cluster_i, avg_dist_other_clusters_i) if max(avg_dist_same_cluster_i, avg_dist_other_clusters_i) > 0 else 0
        silhouettes.append(s_i)

    return np.mean(silhouettes)

In [ ]:
#take part of the sample  for the RAM issues.
sample_size = min(1000, X_clustering.shape[0])
idx = np.random.choice(X_clustering.shape[0], sample_size, replace=False)

internal_score = silhouette(X_clustering[idx], labels[idx])

print(k, "Clusters")
print("Silhouette Score: " ,internal_score)


5 Clusters
Silhouette Score:  0.12992554588312472


### Evaluation of the Internal Metric
The references suggest that bigger than 0.5 ***silhoutte score*** suggests good structure for clustering. We have a really low number for this metric , the most probable reason is that our image features prevent the good structure for clustering, the background of the images could be extracted for better clustering, or may be the HSV values are not enough.

#### External Metric: ***Adjusted Rand Index***
Reference: ***https://www.sciencedirect.com/topics/computer-science/adjusted-rand-index***

***https://medium.com/@samson.sabu/rand-index-ri-vs-adjusted-rand-index-ari-in-k-means-clustering-06c364bdf1ef***

We searched for external metrics; it was hard to understand these metrics compared to the internal ones. AI suggests us the rand index and adjusted rand index. We decided to use the adjusted rand index since it is the improved version of the rand index and considers the effect of accurate clustering randomly by chance.



$$ARI = \frac{\text{Index} - \text{Expected Index}}{\text{Max Index} - \text{Expected Index}}$$



The Adjusted part subtracts the Expected Index (what score we would get if we assigned clusters randomly) from the actual result.





**Confusion Matrix** rows represent the actual fruit types, columns represent the cluster indices.

The metric checks the pairwise probability of having the same cluster index and the same fruit type at the same time.

In [ ]:
def adjusted_rand_index_numpy(y_true, y_pred):
    #Classes and Clusters
    classes, class_idx = np.unique(y_true, return_inverse=True)
    clusters, cluster_idx = np.unique(y_pred, return_inverse=True)
    n = len(y_true)

    #Contingency Table
    contingency = np.zeros((len(classes), len(clusters)))
    for i in range(n):
        contingency[class_idx[i], cluster_idx[i]] += 1

    #Continency Table ---
    print("\n" + "="*60)
    print("CONTINGENCY TABLE (Rows: True Label | Cols: Cluster ID)")
    print("="*60)

    # Header: C0, C1, C2, C3, C4
    header = "True Label".ljust(15) + "".join([f"C{c}".rjust(8) for c in range(len(clusters))])
    print(header)
    print("-" * len(header))

    for i, fruit_name in enumerate(classes):
        counts = contingency[i]
        row_str = "".join([f"{int(c)}".rjust(8) for c in counts])
        print(f"{fruit_name.ljust(15)}{row_str}")

    print("="*60 + "\n")

    #ARI Calculation
    # It calculates the all pairs for clusters and classes with comb_col and comb_row
    # With comb_c, it calculates the aggreements of the cluster and class count
    sum_comb_c = np.sum([n_ij * (n_ij - 1) / 2 for n_ij in contingency.flatten()])
    sum_comb_row = np.sum([ni * (ni - 1) / 2 for ni in contingency.sum(axis=1)])
    sum_comb_col = np.sum([nj * (nj - 1) / 2 for nj in contingency.sum(axis=0)])

    total_comb = n * (n - 1) / 2
    expected_index = (sum_comb_row * sum_comb_col) / total_comb
    max_index = (sum_comb_row + sum_comb_col) / 2

    return (sum_comb_c - expected_index) / (max_index - expected_index)

external_score = adjusted_rand_index_numpy(df['Fruit'].values, labels)
print(f"External Metric (Adjusted Rand Index): {external_score:.4f}")


CONTINGENCY TABLE (Rows: True Label | Cols: Cluster ID)
True Label           C0      C1      C2      C3      C4
-------------------------------------------------------
apple               118     250     191       0      63
banana              135     219     195      44      63
orange              113     191     180      66      58
tangerine           130     211     221      45      61
tomato              130     188     203      45      62

External Metric (Adjusted Rand Index): 0.0020


### Evaluation of the External Metric
The results indicate that our clustering  has failed to find any meaningful patterns related to the fruit types. An Adjusted Rand Index (ARI) of 0.0020 is nearly zero, meaning that the clustering fails, and the reasons are similar to internal metrics.

#### Outlier Detection Method
For every individual data point, the code calculates the Euclidean distance ($L_2$ norm) between that point and the center (centroid) of the cluster it was assigned to.
 Threshold Mechanism: We uses a percentile-based threshold. By setting threshold_percentile=95,we define an outlier as any point whose distance to its centroid is in the top 5% of all distances in the dataset.

In [ ]:
def detect_outliers_by_cluster(X, centroids, labels, threshold_percentile=95):
    #Outliers are the  points that are furthest from their assigned centroids.
    distances = []

    for i in range(len(X)):
        centroid = centroids[labels[i]]
        dist = np.linalg.norm(X[i] - centroid)
        distances.append(dist)

    distances = np.array(distances)

    #distance threshold for the top X% furthest points
    threshold = np.percentile(distances, threshold_percentile)

    outlier_booleans = distances > threshold

    return outlier_booleans, distances

outlier_booleans, dists = detect_outliers_by_cluster(X_clustering, centroids, labels)

# the indices of the top outliers
outlier_indices = np.where(outlier_booleans)[0]
print(f"Total Outliers detected: {len(outlier_indices)}")
print(outlier_indices)

Total Outliers detected: 160
[   9   31   51   64   76   82   89  103  193  224  232  240  304  322
  339  390  420  436  502  514  516  551  598  615  616  621  634  653
  665  675  710  712  796  806  811  818  851  870  885  900  909  916
  931  937  943  963  988 1002 1013 1032 1048 1073 1079 1132 1137 1148
 1160 1200 1213 1236 1242 1254 1286 1287 1293 1305 1307 1311 1313 1358
 1386 1394 1435 1483 1517 1529 1531 1539 1560 1595 1607 1621 1623 1634
 1697 1705 1712 1720 1740 1767 1770 1798 1820 1825 1836 1850 1872 1879
 1893 1894 1898 1902 1912 1935 1940 1955 1957 1963 1964 2046 2080 2145
 2160 2213 2214 2224 2245 2246 2254 2280 2286 2293 2322 2339 2359 2377
 2379 2394 2412 2445 2471 2501 2543 2554 2603 2607 2616 2628 2680 2694
 2731 2778 2822 2836 2847 2852 2887 2923 2928 2934 2974 2992 3011 3052
 3075 3076 3104 3107 3120 3179]
